In [1]:
import os


In [2]:
%pwd
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [6]:


@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        
    

In [13]:
try:
    config = ConfigurationManager()
    print(config)
    
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    print(data_ingestion)
except Exception as e:
    raise e

[2024-05-21 12:14:05,753: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-21 12:14:05,760: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-21 12:14:05,764: INFO: common: created directory at: artifacts]
[2024-05-21 12:14:05,767: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-21 12:14:05,768: INFO: 4124993341: File already exists of size: ~ 7718 KB]
